# Oracle Machine Learning for Python - Data Selection and Manipulation
Oracle Machine Learning for Python (OML4Py), a component of the Oracle Advanced Analytics option to Oracle Database Enterprise Edition, makes the open source Python scripting language and environment ready for the enterprise and big data. Designed for problems involving both large and small volumes of data, Oracle Machine Learning for Python integrates Python with Oracle Database, allowing users to execute Python commands and scripts for statistical, machine learning, and graphical analyses on database tables and views using Python syntax. Many familiar Python functions are overloaded and translate Python functions into SQL for in-database execution, as well as new automated machine learning capabilities. 
![title](img/OML4P_icon.jpg)
In this notebook, we highlight various features of the transparency layer for data selection and manipulation. 

# Connect to Oracle Database
To use OML4Py, first import the package ***oml***. OML4Py supports a variety of connection specification options, including Oracle Wallet. Once connected to an Oracle Database that has OML4Py installed, invoking ***oml.isconnected*** returns true. 

In [1]:
import warnings
warnings.filterwarnings('ignore')

import oml
oml.connect("pyquser","pyquser",
            dsn='(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(service_name=ORCLPDB)))')
oml.isconnected()

True

Create a temporary OML DataFrame proxy object using ***push***. Then, select all rows with...
* the specified column names
* columns whose indices are in the range (1, 4)
* columns of oml.String data type

In [2]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()

x = pd.DataFrame(iris.data, columns = ['SEPAL_LENGTH','SEPAL_WIDTH',
                                       'PETAL_LENGTH','PETAL_WIDTH'])
y = pd.DataFrame(list(map(lambda x: {0: 'setosa', 1: 'versicolor', 
                          2:'virginica'}[x], iris.target)), 
                 columns = ['Species'])
z = pd.concat([x, y], axis=1)

IRIS_TMP = oml.push(z)

In [3]:
IRIS_projected1 = IRIS_TMP[:, ["SEPAL_LENGTH", "PETAL_LENGTH"]]
IRIS_projected1.head(3)

   SEPAL_LENGTH  PETAL_LENGTH
0           5.1           1.4
1           4.9           1.4
2           4.7           1.3

In [4]:
IRIS_projected2 = IRIS_TMP[:, 1:4]
IRIS_projected2.head(3)

   SEPAL_WIDTH  PETAL_LENGTH  PETAL_WIDTH
0          3.5           1.4          0.2
1          3.0           1.4          0.2
2          3.2           1.3          0.2

In [5]:
IRIS_projected3 = IRIS_TMP.select_types(include=[oml.String])
IRIS_projected3.head(3)

  Species
0  setosa
1  setosa
2  setosa

Let's explore various ways of filtering data in-database. 
* select sepal length and petal length where petal length is less than 1.5
* select all rows in which petal length is less than 1.5 or sepal length is 5.0, using the AND and OR conditions in filtering
* select all rows in which petal length is less than 1.5 and sepal length is larger than 5.0

In [6]:
IRIS_filtered1 = IRIS_TMP[IRIS_TMP["PETAL_LENGTH"] < 1.5, 
                                  ["SEPAL_LENGTH", "PETAL_LENGTH"]]
print("Length: ", len(IRIS_filtered1))
IRIS_filtered1.head(3)

Length:  24


   SEPAL_LENGTH  PETAL_LENGTH
0           5.1           1.4
1           4.9           1.4
2           4.7           1.3

In [7]:
IRIS_filtered2 = IRIS_TMP[(IRIS_TMP["PETAL_LENGTH"] < 1.5) | 
                          (IRIS_TMP["SEPAL_LENGTH"] == 5.0), :]
print("Length: ", len(IRIS_filtered2))
IRIS_filtered2.head(3)

Length:  30


   SEPAL_LENGTH  SEPAL_WIDTH  PETAL_LENGTH  PETAL_WIDTH Species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa

In [8]:
IRIS_filtered3 = IRIS_TMP[(IRIS_TMP["PETAL_LENGTH"] < 1.5) & 
                          (IRIS_TMP["SEPAL_LENGTH"] > 5.0), :]
print("Length: ", len(IRIS_filtered3))
IRIS_filtered3.head()

Length:  7


   SEPAL_LENGTH  SEPAL_WIDTH  PETAL_LENGTH  PETAL_WIDTH Species
0           5.1          3.5           1.4          0.2  setosa
1           5.8          4.0           1.2          0.2  setosa
2           5.4          3.9           1.3          0.4  setosa
3           5.1          3.5           1.4          0.3  setosa
4           5.2          3.4           1.4          0.2  setosa

# Using Pandas DataFrame objects
Let's explore operations involving Pandas DataFrames. 

We'll start by creating a temporary table from a Pandas DataFrame and use the ***append*** function. First, append an oml.Float series object to another, then append an oml.DataFrame object to another.

In [9]:
my_df = pd.DataFrame({"id" : [1, 2, 3, 4, 5],
                      "val" : ["a", "b", "c", "d", "e"],
                      "ch" : ["p", "q", "r", "a", "b"],
                      "num" : [4, 3, 6.7, 7.2, 5]})
MY_DF = oml.push(my_df)

print (my_df.dtypes)

num1 = MY_DF['id']
num2 = MY_DF['num']
num1.append(num2)

id       int64
val     object
ch      object
num    float64
dtype: object


[1, 2, 3, 4, 5, 4, 3, 6.7, 7.2, 5]

In [10]:
x = MY_DF[['id', 'val']] 
y = MY_DF[['num', 'ch']] 

print(x.dtypes)
print(y.dtypes)

x.append(y)


id       <class 'oml.core.float.Float'>
val    <class 'oml.core.string.String'>
dtype: object
num      <class 'oml.core.float.Float'>
ch     <class 'oml.core.string.String'>
dtype: object


    id val
0  1.0   a
1  2.0   b
2  3.0   c
3  4.0   d
4  5.0   e
5  4.0   p
6  3.0   q
7  6.7   r
8  7.2   a
9  5.0   b

Next, let's try a few examples using the ***concat*** function.
* create two oml.DataFrame objects and combine the objects column-wise
* create an oml.Float object with the rounded exponential of two times the values in the num column of the oml_frame object, then concatenate it with the oml.DataFrame object y using a new column name
* concatenate object x with multiple objects and turn on automatic name conflict resolution
* concatenate multiple OML data objects and perform customized renaming

In [11]:
from collections import OrderedDict

x = MY_DF[['id', 'val']]
y = MY_DF[['num', 'ch']]
x.concat(y)

   id val  num ch
0   1   a  4.0  p
1   2   b  3.0  q
2   3   c  6.7  r
3   4   d  7.2  a
4   5   e  5.0  b

In [12]:
w = (MY_DF['num']*2).exp().round(decimals=2)
y.concat({'round(exp(2*num))':w})

   num ch  round(exp(2*num))
0  4.0  p            2980.96
1  3.0  q             403.43
2  6.7  r          660003.22
3  7.2  a         1794074.77
4  5.0  b           22026.47

In [13]:
z = MY_DF[:,'id']
x.concat([z, w, y], auto_name=True)

   id val  id3         num  num5 ch
0   1   a    1     2980.96   4.0  p
1   2   b    2      403.43   3.0  q
2   3   c    3   660003.22   6.7  r
3   4   d    4  1794074.77   7.2  a
4   5   e    5    22026.47   5.0  b

In [14]:
x.concat(OrderedDict([('ID',z), ('round(exp(2*num))',w), ('New_',y)]))

   id val  ID  round(exp(2*num))  New_num New_ch
0   1   a   1            2980.96      4.0      p
1   2   b   2             403.43      3.0      q
2   3   c   3          660003.22      6.7      r
3   4   d   4         1794074.77      7.2      a
4   5   e   5           22026.47      5.0      b

Next, we explore merging data. Perform:
* cross join
* left outer join
* right outer join

In [15]:
x = MY_DF[['id', 'val']]
y = MY_DF[['num', 'ch']]

z = x.merge(y)
z

    id_l val_l  num_r ch_r
0      1     a    4.0    p
1      1     a    3.0    q
2      1     a    6.7    r
3      1     a    7.2    a
4      1     a    5.0    b
5      2     b    4.0    p
6      2     b    3.0    q
7      2     b    6.7    r
8      2     b    7.2    a
9      2     b    5.0    b
10     3     c    4.0    p
11     3     c    3.0    q
12     3     c    6.7    r
13     3     c    7.2    a
14     3     c    5.0    b
15     4     d    4.0    p
16     4     d    3.0    q
17     4     d    6.7    r
18     4     d    7.2    a
19     4     d    5.0    b
20     5     e    4.0    p
21     5     e    3.0    q
22     5     e    6.7    r
23     5     e    7.2    a
24     5     e    5.0    b

In [16]:
x.head(4).merge(other=MY_DF[['id', 'num']], on="id", suffixes=['.l','.r'])

   id val.l  num.r
0   1     a    4.0
1   2     b    3.0
2   3     c    6.7
3   4     d    7.2

In [17]:
x.merge(other=y, left_on="id", right_on="num", how="right")

   id_l val_l  num_r ch_r
0   4.0     d    4.0    p
1   3.0     c    3.0    q
2   NaN  None    6.7    r
3   NaN  None    7.2    a
4   5.0     e    5.0    b

Next, try the _drop_ functonality with ***dropna***, ***drop_duplicates***, and ***drop***:
* drop rows with any missing values
* drop rows in which all column values are missing
* drop rows in which any numeric column values are missing
* drop duplicate rows
* drop rows that have the same value in column 'string1' and 'string2'
* drop column 'string2'

In [18]:
my_df2 = pd.DataFrame({'numeric': [1, 1.4, -4, -4, 5.432, None, None],
                   'string1' : [None, None, 'a', 'a', 'a', 'b', None],
                   'string2': ['x', None, 'z', 'z', 'z', 'x', None]})
MY_DF2 = oml.push(my_df2, dbtypes = {'numeric': 'BINARY_DOUBLE',
                                     'string1':'CHAR(1)', 
                                     'string2':'CHAR(1)'})
MY_DF2    

   numeric string1 string2
0    1.000    None       x
1    1.400    None    None
2   -4.000       a       z
3   -4.000       a       z
4    5.432       a       z
5      NaN       b       x
6      NaN    None    None

In [19]:
MY_DF2.dropna(how='any')

   numeric string1 string2
0   -4.000       a       z
1   -4.000       a       z
2    5.432       a       z

In [20]:
MY_DF2.dropna(how='all')

   numeric string1 string2
0    1.000    None       x
1    1.400    None    None
2   -4.000       a       z
3   -4.000       a       z
4    5.432       a       z
5      NaN       b       x

In [21]:
MY_DF2.dropna(how='any', subset=['numeric'])

   numeric string1 string2
0    1.000    None       x
1    1.400    None    None
2   -4.000       a       z
3   -4.000       a       z
4    5.432       a       z

In [22]:
MY_DF2.drop_duplicates()

   numeric string1 string2
0    5.432       a       z
1    1.000    None       x
2   -4.000       a       z
3      NaN       b       x
4    1.400    None    None
5      NaN    None    None

In [23]:
MY_DF2.drop_duplicates(subset=['string1', 'string2'])

   numeric string1 string2
0     -4.0       a       z
1      1.4    None    None
2      1.0    None       x
3      NaN       b       x

In [24]:
MY_DF2.drop('string2')

   numeric string1
0    1.000    None
1    1.400    None
2   -4.000       a
3   -4.000       a
4    5.432       a
5      NaN       b
6      NaN    None

Next, we explore the ***split*** and ***KFold*** function using the _digits_ data set. After creating an OML DataFrame proxy object for the _digits_ data set, 
* sample 20% and 80% of the data
* split the data into four sets
* perform stratification on the target column
* verify that the stratified sampling generates splits in which all of the different categories of digits (digits 0~9) are present in each split
* hash on the target column
* verify that the different categories of digits (digits 0~9) are present in only one of the splits generated by hashing on the category column
* split the data randomly into 4 consecutive folds

In [25]:
import pandas as pd
from sklearn.datasets import load_digits

digits = load_digits()
pd_digits = pd.DataFrame(digits.data,
                         columns=['IMG'+str(i) for i in
                         range(digits['data'].shape[1])])
pd_digits = pd.concat([pd_digits,
                       pd.Series(digits.target,
                                  name = 'target')],
                                  axis = 1)
DIGITS = oml.push(pd_digits)
print("Shape: ", DIGITS.shape)

Shape:  (1797, 65)


In [26]:
splits = DIGITS.split(ratio=(.2, .8), use_hash = False)
print("Split lengths: ", [len(split) for split in splits])

Split lengths:  [351, 1446]


In [27]:
splits = DIGITS.split(ratio = (.25, .25, .25, .25), use_hash = False)
print("Split lengths: ", [len(split) for split in splits])

Split lengths:  [432, 460, 451, 454]


In [28]:
splits = DIGITS.split(strata_cols=['target'])
print("Split lengths: ", [split.shape for split in splits])

Split lengths:  [(1285, 65), (512, 65)]


In [29]:
print("Verify values: ", [split['target'].drop_duplicates().sort_values().pull()
for split in splits])

Verify values:  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]


In [30]:
splits = DIGITS.split(hash_cols=['target'])
print("Split lengths: ", [split.shape for split in splits])

Split lengths:  [(899, 65), (898, 65)]


In [31]:
print("Verify values: ", [split['target'].drop_duplicates().sort_values().pull()
for split in splits])

Verify values:  [[0, 1, 3, 5, 8], [2, 4, 6, 7, 9]]


In [32]:
folds = DIGITS.KFold(n_splits=4)
print("Split lengths: ", [(len(fold[0]), len(fold[1])) for fold in folds])

Split lengths:  [(1352, 445), (1336, 461), (1379, 418), (1325, 472)]


Next, we explore the ***corr*** function. We construct a trivial, highly correlated DataFrame. First, verify that the correlation between column A and B is 1.

In [33]:
import pandas as pd

my_df2 = pd.DataFrame({'A': range(4), 'B': [2*i for i in range(4)]})
MY_DF2 = oml.push(my_df2)
MY_DF2

   A  B
0  0  0
1  1  2
2  2  4
3  3  6

In [34]:
MY_DF2.corr()

A  B
A  1  1
B  1  1

Make a few changes to the data to alter the correlation. 

In [35]:
my_df3 = my_df2

# Change a value to test the change in the computed correlation result
my_df3.loc[2, 'A'] = 1.5

# Change an entry to NaN (not a number) to test the 'skipna' parameter in corr
my_df3.loc[1, 'B'] = None

# Push my_df3 to Oracle Database using the floating point column type 
# because NaNs cannot be used in Oracle numbers
MY_DF3 = oml.push(my_df3, oranumber=False)
MY_DF3

     A    B
0  0.0  0.0
1  1.0  NaN
2  1.5  4.0
3  3.0  6.0

By default, 'skipna' is True.

In [36]:
MY_DF3.corr()

A         B
A  1.000000  0.981981
B  0.981981  1.000000

In [37]:
MY_DF3.corr(skipna=False)

A    B
A  1.0  NaN
B  NaN  1.0

Next, we explore the ***crosstab*** and ***pivot*** functions.

In [38]:
my_df4 = pd.DataFrame({
     'GENDER': ['M', 'M', 'F', 'M', 'F', 'M', 'F', 'F', None, 'F', 'M', 'F'],
     'HAND': ['L', 'R', 'R', 'L', 'R', None, 'L', 'R', 'R', 'R', 'R', 'R'],
     'SPEED': [40.5, 30.4, 60.8, 51.2, 54, 29.3, 34.1, 39.6, 46.4, 12, 25.3, 37.5],
     'ACCURACY': [.92, .94, .87, .9, .85, .97, .96, .93, .89, .84, .91, .95]
    })
MY_DF4 = oml.push(my_df4)

Find the categories that the most entries belonged to.

In [39]:
MY_DF4.crosstab('GENDER', 'HAND').sort_values('count', ascending=False)

  GENDER  HAND  count
0      F     R      5
1      M     L      2
2      M     R      2
3      M  None      1
4      F     L      1
5   None     R      1

For each gender value and across all entries, find the ratio of entries with different hand values.

In [40]:
MY_DF4.crosstab('GENDER', 'HAND', pivot = True, margins = True, normalize = 0)

  GENDER  count_(L)  count_(R)  count_(None)
0   None   0.000000   1.000000      0.000000
1      F   0.166667   0.833333      0.000000
2      M   0.400000   0.400000      0.200000
3    All   0.250000   0.666667      0.083333

Find the mean speed across all gender and hand combinations.

In [41]:
MY_DF4.pivot_table('GENDER', 'HAND', 'SPEED')

  GENDER  mean(SPEED)_(L)  mean(SPEED)_(R)  mean(SPEED)_(None)
0   None              NaN            46.40                 NaN
1      F            34.10            40.78                 NaN
2      M            45.85            27.85                29.3

Find the median accuracy and speed for every gender and hand combination.

In [42]:
MY_DF4.pivot_table('GENDER', 'HAND', aggfunc = oml.DataFrame.median)

  GENDER  median(ACCURACY)_(L)  median(ACCURACY)_(R)  median(ACCURACY)_(None)  \
0   None                   NaN                 0.890                      NaN   
1      F                  0.96                 0.870                      NaN   
2      M                  0.91                 0.925                     0.97   

   median(SPEED)_(L)  median(SPEED)_(R)  median(SPEED)_(None)  
0                NaN              46.40                   NaN  
1              34.10              39.60                   NaN  
2              45.85              27.85                  29.3  

Find the max and min speeds for every gender and hand combination and across all combinations.

In [43]:
MY_DF4.pivot_table('GENDER', 'HAND', 'SPEED', 
               aggfunc = [oml.DataFrame.max, oml.DataFrame.min],
               margins = True)

  GENDER  max(SPEED)_(L)  max(SPEED)_(R)  max(SPEED)_(None)  max(SPEED)_(All)  \
0   None             NaN            46.4                NaN              46.4   
1      F            34.1            60.8                NaN              60.8   
2      M            51.2            30.4               29.3              51.2   
3    All            51.2            60.8               29.3              60.8   

   min(SPEED)_(L)  min(SPEED)_(R)  min(SPEED)_(None)  min(SPEED)_(All)  
0             NaN            46.4                NaN              46.4  
1            34.1            12.0                NaN              12.0  
2            40.5            25.3               29.3              25.3  
3            34.1            12.0               29.3              12.0  

Next, we explore creating new columns using a simple shopping cart data set. Note, that using the transparency layer, all these computations occur in-database. Neither the input nor the result needs to be brought to the client Python engine. 

In [44]:
shopping_cart = pd.DataFrame({
  'Item_name': ['paper_towel', 'ground_pork', 'tofu', 'eggs',
                'pork_loin', 'whole_milk', 'egg_custard'],
  'Item_type': ['grocery', 'meat', 'grocery', 'dairy', 'meat',
                'dairy', 'bakery'],
  'Quantity': [1, 2.6, 4, 1, 1.9, 1, 1],
  'Unit_price': [1.19, 2.79, 0.99, 2.49, 3.19, 2.5, 3.99]
  })
CART = oml.push(shopping_cart)
CART

     Item_name Item_type  Quantity  Unit_price
0  paper_towel   grocery       1.0        1.19
1  ground_pork      meat       2.6        2.79
2         tofu   grocery       4.0        0.99
3         eggs     dairy       1.0        2.49
4    pork_loin      meat       1.9        3.19
5   whole_milk     dairy       1.0        2.50
6  egg_custard    bakery       1.0        3.99

Add a column 'Price' multiplying 'Quantity' with 'Unit_price', rounded to 2 decimal places.

In [45]:
price = CART['Quantity']*(CART['Unit_price'])
print("Type: ", type(price))
CART = CART.concat({'Price': price.round(2)})
CART

Type:  <class 'oml.core.float.Float'>


     Item_name Item_type  Quantity  Unit_price  Price
0  paper_towel   grocery       1.0        1.19   1.19
1  ground_pork      meat       2.6        2.79   7.25
2         tofu   grocery       4.0        0.99   3.96
3         eggs     dairy       1.0        2.49   2.49
4    pork_loin      meat       1.9        3.19   6.06
5   whole_milk     dairy       1.0        2.50   2.50
6  egg_custard    bakery       1.0        3.99   3.99

Count the pattern 'egg' in the 'Item_name' column.

In [46]:
egg_pattern = CART['Item_name'].count_pattern('egg')
print("Type: ", type(egg_pattern))
CART.concat({'Egg_pattern': egg_pattern})

Type:  <class 'oml.core.float.Float'>


     Item_name Item_type  Quantity  Unit_price  Price  Egg_pattern
0  paper_towel   grocery       1.0        1.19   1.19            0
1  ground_pork      meat       2.6        2.79   7.25            0
2         tofu   grocery       4.0        0.99   3.96            0
3         eggs     dairy       1.0        2.49   2.49            1
4    pork_loin      meat       1.9        3.19   6.06            0
5   whole_milk     dairy       1.0        2.50   2.50            0
6  egg_custard    bakery       1.0        3.99   3.99            1

Find the start index of substring 'pork' in the 'Item_name' column.

In [47]:
pork_startInd = CART['Item_name'].find('pork')
print("Type: ", type(pork_startInd))
CART.concat({'Pork_startInd': pork_startInd})

Type:  <class 'oml.core.float.Float'>


     Item_name Item_type  Quantity  Unit_price  Price  Pork_startInd
0  paper_towel   grocery       1.0        1.19   1.19             -1
1  ground_pork      meat       2.6        2.79   7.25              7
2         tofu   grocery       4.0        0.99   3.96             -1
3         eggs     dairy       1.0        2.49   2.49             -1
4    pork_loin      meat       1.9        3.19   6.06              0
5   whole_milk     dairy       1.0        2.50   2.50             -1
6  egg_custard    bakery       1.0        3.99   3.99             -1

Check whether items are of grocery category.

In [48]:
is_grocery=CART['Item_type']=='grocery'
print("Type: ", type(is_grocery))
CART.concat({'Is_grocery': is_grocery})

Type:  <class 'oml.core.boolean.Boolean'>


     Item_name Item_type  Quantity  Unit_price  Price  Is_grocery
0  paper_towel   grocery       1.0        1.19   1.19        True
1  ground_pork      meat       2.6        2.79   7.25       False
2         tofu   grocery       4.0        0.99   3.96        True
3         eggs     dairy       1.0        2.49   2.49       False
4    pork_loin      meat       1.9        3.19   6.06       False
5   whole_milk     dairy       1.0        2.50   2.50       False
6  egg_custard    bakery       1.0        3.99   3.99       False

Calculate the length of item names.

In [49]:
name_length=CART['Item_name'].len()
print("Type: ", type(name_length))
CART.concat({'Name_length': name_length})

Type:  <class 'oml.core.float.Float'>


     Item_name Item_type  Quantity  Unit_price  Price  Name_length
0  paper_towel   grocery       1.0        1.19   1.19           11
1  ground_pork      meat       2.6        2.79   7.25           11
2         tofu   grocery       4.0        0.99   3.96            4
3         eggs     dairy       1.0        2.49   2.49            4
4    pork_loin      meat       1.9        3.19   6.06            9
5   whole_milk     dairy       1.0        2.50   2.50           10
6  egg_custard    bakery       1.0        3.99   3.99           11

Use a few other function to modify the Price column. 

In [50]:
print("Ceiling: ", CART['Price'].ceil())
print("Floor: ", CART['Price'].floor())
print("Exponential: ", CART['Price'].exp().round(3))
print("logarithm: ", CART['Price'].log().round(3))
print("Square Root: ", CART['Price'].sqrt().round(3))

Ceiling:  [2, 8, 4, 3, 7, 3, 4]
Floor:  [1, 7, 3, 2, 6, 2, 3]
Exponential:  [3.287, 1408.105, 52.457, 12.061, 428.375, 12.182, 54.055]
logarithm:  [0.174, 1.981, 1.376, 0.912, 1.802, 0.916, 1.384]
Square Root:  [1.091, 2.693, 1.99, 1.578, 2.462, 1.581, 1.997]


Next, using a modified version of the _iris_ data set, explore ***sort_values*** and ***tail*** functions. 

We modify the dataset by replacing a few entries with NaNs to test how the _na_position_ parameter works in the ***sort_values*** method.

In [51]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()

x = pd.DataFrame(iris.data, columns = ['Sepal_Length', 'Sepal_Width',
                                       'Petal_Length', 'Petal_Width'])
y = pd.DataFrame(list(map(lambda x: {0: 'setosa', 1: 'versicolor',
                                     2:'virginica'}[x], iris.target)), 
                 columns = ['Species'])
                   
x['Sepal_Width'].replace({3.5: None}, inplace=True)
x['Petal_Length'].replace({1.5: None}, inplace=True)
x['Petal_Width'].replace({2.3: None}, inplace=True)

iris_df = pd.concat([x, y], axis=1)
IRIS2 = oml.push(iris_df)

Sort the data set first by Sepal_Length then by Sepal_Width in descending order and display the first 5 rows of the sorted result.

In [52]:
IRIS2.sort_values(by = ['Sepal_Length', 'Sepal_Width'],
                  ascending=False).head()

   Sepal_Length  Sepal_Width  Petal_Length  Petal_Width    Species
0           7.9          3.8           6.4          2.0  virginica
1           7.7          3.8           6.7          2.2  virginica
2           7.7          3.0           6.1          NaN  virginica
3           7.7          2.8           6.7          2.0  virginica
4           7.7          2.6           6.9          NaN  virginica

Display the last 5 rows of the data set.

In [53]:
IRIS2.tail()

   Sepal_Length  Sepal_Width  Petal_Length  Petal_Width    Species
0           6.7          3.0           5.2          NaN  virginica
1           6.3          2.5           5.0          1.9  virginica
2           6.5          3.0           5.2          2.0  virginica
3           6.2          3.4           5.4          NaN  virginica
4           5.9          3.0           5.1          1.8  virginica

Sort the last 5 rows of the iris data set first by Petal_Length then by Petal_Width. By default, rows with NaNs are placed after the other rows when the sort keys are the same.

In [54]:
IRIS2.tail().sort_values(by = ['Petal_Length', 'Petal_Width'])

   Sepal_Length  Sepal_Width  Petal_Length  Petal_Width    Species
0           6.3          2.5           5.0          1.9  virginica
1           5.9          3.0           5.1          1.8  virginica
2           6.5          3.0           5.2          2.0  virginica
3           6.7          3.0           5.2          NaN  virginica
4           6.2          3.4           5.4          NaN  virginica

Sort the last 5 rows of the IRIS data set first by Petal_Length and then by Petal_Width. When the values in these two columns are the same, place the row with a NaN before the other row.

In [55]:
IRIS2.tail().sort_values(by = ['Petal_Length', 'Petal_Width'], 
                                 na_position = 'first')

   Sepal_Length  Sepal_Width  Petal_Length  Petal_Width    Species
0           6.3          2.5           5.0          1.9  virginica
1           5.9          3.0           5.1          1.8  virginica
2           6.7          3.0           5.2          NaN  virginica
3           6.5          3.0           5.2          2.0  virginica
4           6.2          3.4           5.4          NaN  virginica

Next, we explore column creation/renaming, describing data statistics, and computing a variety of statistics. 

In [56]:
my_df4 = pd.DataFrame({'numeric_col': [1, 1.4, -4, 3.145, 5, None],
                       'string_col' : [None, None, 'a', 'a', 'a', 'b'],
                       'bytes_col' : [b'a', b'b', b'c', b'c', b'd', b'e']})

MY_DF4 = oml.push(my_df4, dbtypes = {'numeric_col': 'BINARY_DOUBLE',
                                     'string_col':'CHAR(1)', 
                                     'bytes_col':'RAW(1)'})
MY_DF4

   numeric_col string_col bytes_col
0        1.000       None      b'a'
1        1.400       None      b'b'
2       -4.000          a      b'c'
3        3.145          a      b'c'
4        5.000          a      b'd'
5          NaN          b      b'e'

Combine a Boolean column with MY_DF4.

In [57]:
oml_bool = MY_DF4['numeric_col'] > 3

MY_DF4 = MY_DF4.concat(oml_bool)
MY_DF4.rename({'COL4':'boolean_col'})

   numeric_col string_col bytes_col  boolean_col
0        1.000       None      b'a'        False
1        1.400       None      b'b'        False
2       -4.000          a      b'c'        False
3        3.145          a      b'c'         True
4        5.000          a      b'd'         True
5          NaN          b      b'e'         True

Use the ***describe*** function to compute a variety of statistics on all columns. Then, exclude the Float columns. 

In [58]:
MY_DF4.describe(include='all')

numeric_col string_col bytes_col boolean_col
count      5.000000          4         6           6
unique          NaN          2         5           2
top             NaN          a      b'c'       False
freq            NaN          3         2           3
mean       1.309000        NaN       NaN         NaN
std        3.364655        NaN       NaN         NaN
min       -4.000000        NaN       NaN         NaN
25%        1.000000        NaN       NaN         NaN
50%        1.400000        NaN       NaN         NaN
75%        3.145000        NaN       NaN         NaN
max        5.000000        NaN       NaN         NaN

In [59]:
MY_DF4.describe(exclude=[oml.Float])

string_col bytes_col boolean_col
count           4         6           6
unique          2         5           2
top             a      b'c'       False
freq            3         2           3

Compute the sum of values in each Float or Boolean column.

In [60]:
MY_DF4.sum()

numeric_col    6.545
boolean_col    3.000
dtype: float64

Find the cumulative sum of values in each Float or Boolean column after MY_DF4 is sorted by the bytes column in descending order. Then, compute skew, median, and kurtosis for float columns. 

In [61]:
MY_DF4.cumsum(by = 'bytes_col', ascending = False)

   numeric_col  boolean_col
0          NaN            1
1        5.000            2
2        1.000            2
3        4.145            3
4        5.545            3
5        6.545            3

In [62]:
MY_DF4.skew()

numeric_col   -0.683838
dtype: float64

In [63]:
MY_DF4.median()

numeric_col    1.4
dtype: float64

In [64]:
MY_DF4.kurtosis()

numeric_col   -0.582684
dtype: float64

Next, explore the ***boxplot*** and ***hist*** functions using the _wine_ data set. The statistics supporting these plots are computed in-database, so transfer of data or memory limitations are avoided. 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine

wine = load_wine()
wine_data = pd.DataFrame(wine.data, columns = wine.feature_names)
WINE = oml.push(wine_data)

oml.graphics.boxplot(WINE[:,8:12], showmeans=True, 
                     meanline=True, patch_artist=True, 
                     labels=WINE.columns[8:12])
plt.title('Distribution of Wine Attributes')
plt.show()

In [ ]:
oml.graphics.hist(WINE['proline'], bins=10, color='red',
                  linestyle='solid', edgecolor='black')
plt.title('Proline content in Wine')
plt.xlabel('proline content')
plt.ylabel('# of wine instances')
plt.show()

In [ ]:
oml.disconnect()

<img src="img/Oracle-sm.jpg">